In [61]:
import numpy as np
import pandas as pd
import re #biểu thức chính quy
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from underthesea import chunk
from underthesea import text_normalize
import swifter

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

Đọc dữ liệu

In [62]:
real_data = pd.read_csv('data/RealNewsfrom_NewsData_io_UPDATED.csv', encoding='utf-8')
fake_data = pd.read_csv('data/FakeNews.csv', encoding='utf-8')

Xác định và chuẩn bị dữ liệu

In [63]:
real_data['label'] = 0 #real
fake_data['label'] = 1 #fake

# Kết hợp hai DataFrame thành một DataFrame lớn
data = pd.concat([real_data, fake_data], ignore_index=True)
data = data.fillna('')

Gộp các trường title, content, creator lại thành 1 trường text (1 cột trong dataframe)

In [64]:
data['text'] = data['title']+ ' ' + data['content'] + ' ' + data['creator']

Mở tệp dữ liệu temp_data_noClean.csv và kiểm tra

In [65]:
data_check = pd.read_csv('data/temp_data_noClean.csv', encoding='utf-8')
print(data['text'])

0       Ông Tucker Carlson đưa ra lời cảnh báo tới ông...
1       Đức điều động binh sĩ tới Lithuania trong đợt ...
2       Thừa nhận mới nhất của ĐCSTQ về tình hình dịch...
3       Nghe tiếng khóc của đứa trẻ sơ sinh, biết trướ...
4       Con trai Trương Tấn Sang bị đẩy ra khỏi TP.HCM...
                              ...                        
1787     Ở Việt Nam bị cấm tuyệt đối không được kết hô...
1788     Người Dân trên toàn trái đất đi sơ tán vì sắp...
1789     Tất cả những tin tức trên Facebookn đều là gi...
1790     Đưa tin thất thiệt mang nhiều hệ quả rất xấu ...
1791              Đa cấp là ngành thiết yếu của Việt Nam 
Name: text, Length: 1792, dtype: object


Tối ưu dữ liệu nhập vào tránh phải chuẩn hóa các bài báo cũ:
- Bước 1: Ghép (merge) hai DataFrame data và data_check dựa trên cột 'text' với kiểu ghép là 'outer' (ghép toàn bộ).
- Bước 2: Lọc những hàng chỉ xuất hiện trong data mà không xuất hiện trong data_check.
- Bước 3: Kiểm tra missing_rows_data_in_datacheck có giá trị được truyền vào hay không:
    + Nếu không phát hiện không có giá trị nào được truyền vào thì trả về 1 dataframe rỗng chứa duy nhất các trường mặc định của data
    + Nếu có giá trị được truyền vào missing_rows_data_in_datacheck thức hiện các mục:
In ra result_value để kiểm tra

In [66]:

merged_data_datacheck = pd.merge(data, data_check, on='text', how='outer', indicator=True) # 

# Lọc các hàng chỉ xuất hiện trong data
missing_rows_data_in_datacheck = merged_data_datacheck[merged_data_datacheck['_merge'] == 'left_only']

# Kiểm tra xem missing_rows_data_in_datacheck có tồn tại hay không
if missing_rows_data_in_datacheck.empty:
    result_value = pd.read_csv('data/temp.csv')
else:
    # Nếu có, thực hiện các bước tiếp theo
    data_check = data.copy()                                        #-> copy toàn bộ dữ liệu từ dataframe của data để update data_check (thêm những bài báo mới)
    data_check.to_csv('data/temp_data_noClean.csv', index=False)    #-> đẩy data_check thành 1 file csv để thay thế file cũ (chứa những bài báo cũ lần đã cập nhật)
    temp = missing_rows_data_in_datacheck.copy()                    #-> copy những bài báo data có mà data_check không có vào biến temp
    temp.rename(columns={'label_x': 'label'}, inplace=True)         #-> thay đổi tên trường trong dataframe temp cho phù hợp với dữ liệu đúng
    selected_columns = ['text', 'label']                            #-> tạo một biến mang 2 trường là text và label
    temp_selected = temp[selected_columns]                          #-> tạo 1 dataframe temp_selected chỉ mang 2 trường là text và label của temp
    result_value = temp_selected                                    #-> trả về giá trị cuối là result_value

# In kết quả hoặc sử dụng result_value theo nhu cầu
print(result_value)


                                                   text  label
984   Truy tìm đối tượng bỏ trốn sau khi gây ra tai ...      0
985   Chùa Ba Vàng gỡ thông tin, hình ảnh liên quan ...      0
986   Ảnh cưới full HD của Quang Hải - Chu Thanh Huy...      0
987   Dự tiệc cuối năm, nhiều lái xe ở TP.HCM vi phạ...      0
988   Thu Phương, Anh Tú hăng say tổng duyệt Chung k...      0
...                                                 ...    ...
1046  Hiện trạng khu đất 'vàng' 20.000 m2 ở Nha Tran...      0
1047  Bị chữa răng suốt 6 giờ, bệnh nhân kiện nha sĩ...      0
1048  Bi kịch gia đình sau câu thách thức của người ...      0
1049  Lập 280 công ty để bán hóa đơn VAT hơn 25.000 ...      0
1050  Chủ nhà hàng ở Sài Gòn tuyển 200 nhân viên phụ...      0

[67 rows x 2 columns]


Chuẩn hóa, Phân tích từ

In [67]:
def word(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    chunks = chunk(text)
    filtered_chunks = [chunk[0] for chunk in chunks if chunk[2] != 'O']
    result_sentence = ' '.join(filtered_chunks)
    return result_sentence

result_value.loc[:, 'text'] = result_value['text'].swifter.apply(word).copy()

Pandas Apply:   0%|          | 0/67 [00:00<?, ?it/s]

In [68]:
data_have_clean = pd.read_csv('data/temp_data_noClean.csv', encoding='utf-8')

In [69]:
data = pd.concat([data_have_clean, result_value[['text', 'label']]], ignore_index=True)
data = data.fillna('')
print(data['text'])

0       Ông Tucker Carlson đưa ra lời cảnh báo tới ông...
1       Đức điều động binh sĩ tới Lithuania trong đợt ...
2       Thừa nhận mới nhất của ĐCSTQ về tình hình dịch...
3       Nghe tiếng khóc của đứa trẻ sơ sinh, biết trướ...
4       Con trai Trương Tấn Sang bị đẩy ra khỏi TP.HCM...
                              ...                        
1854    hiện trạng khu đất vàng 20000 m2 ở nha trang đ...
1855    bị chữa răng suốt 6 giờ bệnh nhân kiện nha sĩ ...
1856    bi kịch gia đình sau câu thách thức của người ...
1857    lập 280 công ty để bán hóa đơn vat hơn 25000 t...
1858    chủ nhà hàng ở sài gòn tuyển 200 nhân viên phụ...
Name: text, Length: 1859, dtype: object


In [70]:
data.isnull().sum()

article_id     0
title          0
link           0
creator        0
description    0
content        0
source_id      0
category       0
label          0
text           0
dtype: int64

In [71]:
X_train, X_test, y_train, y_test = train_test_split(
    data['text'],
    data['label'],
    test_size=0.2,
    random_state=42
    )

In [72]:
vectorized = TfidfVectorizer()
param_grid = {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10], 'gamma': ['scale', 'auto']}
X_train_tfidf = vectorized.fit_transform(X_train)
X_test_tfidf = vectorized.transform(X_test)

In [73]:
svm_model = SVC()
grid_search = GridSearchCV(svm_model, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train)

GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf']})

In [74]:
print('Best hyperparameters:', grid_search.best_params_)


Best hyperparameters: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}


In [75]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_tfidf)


In [76]:
y_pred

array([0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,

In [77]:
import pickle
with open('temp/vectorized.pkl', 'wb') as f:
    pickle.dump(vectorized, f)  
with open('temp/svm_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)
with open('temp/word_clean.pkl', 'wb') as f:
    pickle.dump(word, f)

In [79]:
print(len(y_test), len(y_pred))

372 372


In [80]:
print(f"Số hàng trong X_train: {len(X_train)}")
print(f"Số phần tử trong y_train: {len(y_train)}")
print(f"Số hàng trong X_test: {len(X_test)}")
print(f"Số phần tử trong y_test: {len(y_test)}")

Số hàng trong X_train: 1487
Số phần tử trong y_train: 1487
Số hàng trong X_test: 372
Số phần tử trong y_test: 372


In [81]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

accuracy = accuracy_score(y_test, y_pred)
print(f'Độ Chính Xác: {accuracy}')
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc}")

print("-------------------------------------------------------------------------------------")
# Dự đoán trên tập kiểm tra
y_pred = best_model.predict(X_test_tfidf)

# Tính báo cáo đánh giá
report = classification_report(y_test, y_pred)

# In báo cáo đánh giá
print("Classification Report:")
print(report)

print("-------------------------------------------------------------------------------------")

# Dự đoán trên tập kiểm tra
y_pred = best_model.predict(X_test_tfidf)

# Tính ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_test, y_pred)

# In ma trận nhầm lẫn
print("Confusion Matrix:")
print(conf_matrix)


Độ Chính Xác: 0.9489247311827957
ROC-AUC Score: 0.945384310013131
-------------------------------------------------------------------------------------
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       229
           1       0.94      0.93      0.93       143

    accuracy                           0.95       372
   macro avg       0.95      0.95      0.95       372
weighted avg       0.95      0.95      0.95       372

-------------------------------------------------------------------------------------
Confusion Matrix:
[[220   9]
 [ 10 133]]
